# Set 1

In [26]:
%pip install numpy scipy

Note: you may need to restart the kernel to use updated packages.


In [27]:
import base64
from string import ascii_letters, printable
from collections import Counter
import numpy as np
import scipy

In [28]:
hamlet = open("hamlet.txt").read()

def letter_freq(text) -> dict:
    return dict(Counter([x.lower() for x in text if x in ascii_letters]))

hamlet_freq = letter_freq(hamlet)
hamlet_freq_arr = np.array(list(hamlet_freq.values()))

## Challenge 1: Convert hex to base64

In [33]:
string = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
bs = bytes.fromhex(string)
b64 = base64.b64encode(bs)
b64

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

## Challenge 2: Fixed XOR

In [34]:
a = bytes.fromhex("1c0111001f010100061a024b53535009181c")
b = bytes.fromhex("686974207468652062756c6c277320657965")

bytes([a ^ b for a, b in zip(a, b)]).hex()

'746865206b696420646f6e277420706c6179'

## Challenge 3: Single-byte XOR cipher

In [58]:
import scipy.stats


string = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
bs = bytes.fromhex(string)

strings = []
for ch in ascii_letters:
    new = bytes([b ^ ord(ch) for b in bs])
    strings.append(new)

def score(bstr):
    if any(chr(x) not in printable or chr(x) in "{}()" for x in bstr):
        return -1
    freq = letter_freq(bstr.decode("ascii").lower())
    freq_arr = np.array(list(freq.values()))
    legit_text_statistic = len(bstr) - sum([1 for x in bstr if chr(x) not in ascii_letters + ".,?!' "])
    ks_test_2 = scipy.stats.ks_2samp(hamlet_freq_arr, freq_arr).pvalue ** 2
    return legit_text_statistic / ks_test_2
    

strings.sort(key=score, reverse=True)
strings[0]

b"Cooking MC's like a pound of bacon"

## Challenge 4: Detect single-character XOR

In [56]:
bs = [bytes.fromhex(s) for s in open("challenge-data/4.txt").readlines()]

guesses = []
for string in bs:
    for ch in range(255):
        new = bytes([b ^ ch for b in string])
        s = score(new)
        guesses.append((s, new))
guesses.sort(reverse=True)

guesses[0][1]

b'Now that the party is jumping\n'